In [ ]:
#import modules

import dask.array as da
from dask_ml.model_selection import train_test_split
import dask.dataframe as dd
from dask_ml.decomposition import PCA   
import matplotlib.pyplot as plt
from dask.distributed import Client
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

In [ ]:
#import data matrix

fpath = 'C:\processed_data_sensor_2\data_matricies\svd_data_matrix\week3_third_cycle\demeaned_data_matrix_third_cycle_week_3'
ddf = dd.read_csv(fpath, sep=',', header=0)

ddf = ddf.rename(columns={'Unnamed: 0': 'temperature'});

In [ ]:
client = Client(n_workers=2, threads_per_worker=1, memory_limit='20GB')


In [ ]:
col_list = ddf.columns
col_list[0]

In [ ]:

# slice the ddf into X and y
X = ddf[col_list[1:]]
y = ddf[col_list[0]]; 

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=True, random_state=42)

X_train_array = X_train.to_dask_array(lengths=True)
X_test_array = X_test.to_dask_array(lengths=True)


In [ ]:
temp = y.compute().to_numpy()

plt.plot(temp)

In [ ]:
# Create a Dask PCA object
n_comps = 20
pca = PCA(n_components=n_comps, svd_solver='randomized')

# Fit the PCA model
pca.fit_transform(X_train_array) 
pca.transform(X_test_array)

In [ ]:
# Get explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

In [ ]:
# filename and path for figure
export_name = 'c:/sams//saved_data/sensor_2_week_3_third_cycle_screeplot.png'


# Create scree plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.xticks(range(1, len(explained_variance_ratio) + 1))
plt.grid()
plt.savefig(export_name, dpi=700)
plt.show()


In [ ]:

print(pca.singular_values_)

In [ ]:
# =============================================================================
# save pca modes
# =============================================================================

pca_export = 'c:/sams//saved_data/sensor_2_week_3_third_cycle_pca_modes.csv'

df_pca = pd.DataFrame(pca.components_)

df_pca.to_csv(pca_export)

In [ ]:
# =============================================================================
# compute loadings
# =============================================================================

loadings_train = (X_train_array@pca.components_.T).compute()
loadings_test = (X_test_array@pca.components_.T).compute()
y_train_comp = y_train.compute()

# wavelength labels for x axis
x_label =ddf.columns[1:].astype('float')


In [ ]:

#save figs for each mode
export_name = 'c:/sams//saved_data/sensor_2_week_3_third_cycle_eigenmode_{}.png'


# plot each of the components
for i in range(n_comps):
    plt.plot(x_label, pca.components_.T[:,i]); 
    plt.title('eigenmode {}'.format(i))
    plt.xlabel('Wavelength (nm)')
    plt.savefig(export_name.format(i), dpi=700)
    plt.show()

#save figs for each mode's laoding 
export_name_loadings = 'c:/sams/saved_data/sensor_2_week_3_third_cycle_eigenmode_{}_loading.png'

# make loading plots for each mode
for i in range(n_comps):
    plt.plot(loadings_train[:, i])
    plt.title('loading for {}-th eigemode'.format(i))
    plt.savefig(export_name_loadings.format(i), dpi=700)
    plt.show()


# plot components against first mode (zeroth mode)

for i in range(n_comps):
    if i ==0:
        pass
    else:
        plt.plot(loadings_train[:,0],loadings_train[:,i], '*' )
        plt.xlabel('first mode')
        plt.ylabel('mode {}'.format(i))
        plt.savefig('c:/sams/saved_data/sensor_2_week_3_eigenmode_{}_against_first_mode.png'.format(i))
        plt.show()

In [ ]:
y_pred_test_comp = y_test.compute()

In [ ]:
# =============================================================================
# test linear regression without reseting y index
# =============================================================================
lnr = LinearRegression()
x_t = loadings_train[:, 0:7]
lnr.fit(x_t, y_train)
y_pd = lnr.predict(x_t)
tr = np.sqrt(mean_squared_error(y_train, y_pd))
print(f'training error with 7 comps is {tr:3f}')
y_test_pd = lnr.predict(loadings_test[:, 0:7])
tr_t = np.sqrt(mean_squared_error(y_test, y_test_pd))
print(f'testing error with 7 comps is {tr_t:3f}')

#y_pred_test_comp = y_test_pd.compute()


plt.plot(y_pred_test_comp, y_test_pd, 'x')
plt.title('Linear regression')
plt.xlabel('Measured Temperature (deg C)')
plt.ylabel('Predicted Temperature (deg C)')
#plt.savefig('c:/sams/saved_data/sensor_2_week_3_third_cycle_LINEAR_testing.png', dpi = 700)
plt.show()

plt.plot(y_pred_test_comp, y_pred_test_comp-y_test_pd, 'x')
plt.title('LINEAR regression')
plt.xlabel('Measured Temperature (deg C)')
plt.ylabel('Predicted Temperature (deg C)')
#plt.savefig('c:/sams/saved_data/sensor_2_week_3_third_cycle_LINEAR_testing_residual.png', dpi = 700)
plt.show()


In [ ]:

# =============================================================================
# linear regression search for optimal n_comps
# =============================================================================

ncomps =[]
coefs = []
training_error= []
testing_error =[]

#y_ = y_train_comp.reset_index()['temperature']
#y__ = y_test.compute().reset_index()['temperature']
ln = LinearRegression()

for i in range(n_comps):
    if i ==0:
        pass
    else:
        ncomps.append(i)
        x_ = (loadings_train[:, 0:i])
        ln.fit(x_, y_train)
        coefs.append(ln.coef_)
        y_pred_training = ln.predict(x_)
        tr_error = np.sqrt(mean_squared_error(y_train, y_pred_training))
        training_error.append(tr_error)
        print(f'training error with {i} comps is {tr_error:3f}')
        y_pred_testing =  ln.predict(loadings_test[:, 0:i])
        tt_error = np.sqrt(mean_squared_error(y_test, y_pred_testing))
        testing_error.append(tt_error)
        print(f'testing error with {i} comps is {tt_error:3f}')

df_ = pd.DataFrame(list(zip(ncomps,training_error, testing_error, coefs)))
df_.columns = ['components', 'training_error', 'testing_error', 'coefs']

plt.plot(df_.components, df_.training_error)
plt.plot(df_.components, df_.testing_error)
plt.show()

df_.to_csv('n_comp_evaluation_third_cycle.csv')


In [ ]:

# =============================================================================
# Lasso regression
# =============================================================================

x_ = (loadings_train[:, 0:7])
y_ = y_train#_comp.reset_index()['temperature']
y__ = y_test#.compute().reset_index()['temperature']
ls = Lasso(alpha = 0.1)
ls.fit(x_, y_)
y_pred_training = ls.predict(x_)
print(f'training error is {mean_squared_error(y_train, y_pred_training):3f}')

y_pred_testing =  ls.predict(loadings_test[:, 0:7])
print(f'testing error is {mean_squared_error(y__, y_pred_testing):3f}')

plt.plot(y_pred_test_comp, y_pred_testing, 'x')
plt.title('LASSO regression')
plt.xlabel('Measured Temperature (deg C)')
plt.ylabel('Predicted Temperature (deg C)')
plt.savefig('c:/sams/saved_data/sensor_2_week_3_third_cycle_LASSO_testing.png', dpi = 700)
plt.show()

plt.plot(y_pred_test_comp, y_pred_test_comp-y_pred_testing, 'x')
plt.title('LASSO regression')
plt.xlabel('Measured Temperature (deg C)')
plt.ylabel('Predicted Temperature (deg C)')
plt.savefig('c:/sams/saved_data/sensor_2_week_3_third_cycle_LASSO_testing_residual.png', dpi = 700)
plt.show()




# set hyperparameter search for alpha

alphas = np.array([1e-6, 1e-5,1e-4, 1e-3, 1e-2, 0.005, 0.1, 0.5, 1])

alphas_l =[]
training_error_l= []
testing_error_l = []
coef_l = []

for a in alphas:
    alphas_l.append(a)
    ls_ = Lasso(alpha = a)
    ls_.fit(x_, y_)
    y_pred_training = ls_.predict(x_)
    training_error_l.append(mean_squared_error(y_train, y_pred_training))
    y_pred_testing =  ls_.predict(loadings_test[:, 0:7])
    testing_error_l.append(mean_squared_error(y__, y_pred_testing))
    coef_l.append(ls_.coef_)

plt.plot(alphas_l, training_error_l[:], 'x')
plt.title('LASSO regression optimization')
plt.xlabel('alphas')
plt.ylabel('training_error')
plt.savefig('c:/sams/saved_data/sensor_2_week_3_third_cycle_LASSO_alphas_training_error.png', dpi = 700)
plt.show()

plt.plot(alphas_l, testing_error_l, 'x')
plt.title('LASSO regression optimization testing error')
plt.xlabel('alphas')
plt.ylabel('testing_error')
plt.savefig('c:/sams/saved_data/sensor_2_week_3_third_cycle_LASSO_alphas_testing_error.png', dpi = 700)
plt.show()


In [ ]:

# =============================================================================
# Ridge regression
# =============================================================================

x_ = (loadings_train[:, 0:7])
y_ = y_train#_comp.reset_index()['temperature']
y__ = y_test#.compute().reset_index()['temperature']
lrd = Ridge(alpha = 0.1)
lrd.fit(x_, y_)
y_pred_training = lrd.predict(x_)
print(f'training error is {mean_squared_error(y_train, y_pred_training):3f}')

y_pred_testing =  lrd.predict(loadings_test[:, 0:7])
print(f'testing error is {mean_squared_error(y__, y_pred_testing):3f}')



plt.plot(y_pred_test_comp, y_pred_testing, 'x')
plt.title('RIDGE regression')
plt.xlabel('Measured Temperature (deg C)')
plt.ylabel('Predicted Temperature (deg C)')
plt.savefig('c:/sams/saved_data/sensor_2_week_3_third_cycle_RIDGE_testing.png', dpi = 700)
plt.show()

plt.plot(y_pred_test_comp, y_pred_test_comp-y_pred_testing, 'x')
plt.title('RIDGE regression')
plt.xlabel('Measured Temperature (deg C)')
plt.ylabel('Predicted Temperature (deg C)')
plt.savefig('c:/sams/saved_data/sensor_2_week_3_third_cycle_RIDGE_testing_residual.png', dpi = 700)
plt.show()





# set hyperparameter search for alpha


alphas_ridge =[]
training_error_ridge= []
testing_error_ridge = []

for a in alphas:
    alphas_ridge.append(a)
    ridge_reg = Ridge(alpha = a)
    ridge_reg.fit(x_, y_)
    y_pred_training = ridge_reg.predict(x_)
    training_error_ridge.append(mean_squared_error(y_train, y_pred_training))
    y_pred_testing =  ridge_reg.predict(loadings_test[:, 0:7])
    testing_error_ridge.append(mean_squared_error(y__, y_pred_testing))

plt.plot(alphas_ridge, training_error_ridge, 'x')
plt.title('RIDGE regression optimization')
plt.xlabel('alphas')
plt.ylabel('training_error')
plt.savefig('c:/sams/saved_data/sensor_2_week_3_third_cycle_RIDGE_alphas_training_error.png', dpi = 700)
plt.show()

plt.plot(alphas_ridge, testing_error_ridge, 'x')
plt.title('RIDGE regression optimization testing error')
plt.xlabel('alphas')
plt.ylabel('testing_error')
plt.savefig('c:/sams/saved_data/sensor_2_week_3_third_cycle_RIDGE_alphas_testing_error.png', dpi = 700)
plt.show()





In [ ]:
client.close()